In [48]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import models
import importlib
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from matplotlib import pyplot as plt
K = tf.keras.backend
import string

In [34]:
importlib.reload(models)

<module 'models' from '/Users/varun/School/CS182/finalproj/CS182-Spring2020-NLP-Project/models.py'>

In [82]:
df = pd.read_csv("dataset-binary.csv")
df = df.dropna()
train_df = df.iloc[:int(len(df) * .8)]
test_df = df.iloc[int(len(df) * .8):]

In [83]:
train_texts = train_df["text"].values
train_stars = train_df[["stars_1.0", "stars_2.0", "stars_3.0", "stars_4.0", "stars_5.0"]].values
test_texts = test_df["text"].values
test_stars = test_df[["stars_1.0", "stars_2.0", "stars_3.0", "stars_4.0", "stars_5.0"]].values

In [88]:
def get_star_from_one_hot(star):
    return np.argmax(star) + 1

In [89]:
stars_un_one_hot = np.array([get_star_from_one_hot(i) for i in train_stars])

In [90]:
one_stars = []
two_stars = []
three_stars = []
four_stars = []
five_stars = []
for i in range(len(train_texts)):
    if stars_un_one_hot[i] == 1:
        one_stars.append((train_texts[i], 1))
    if stars_un_one_hot[i] == 2:
        two_stars.append((train_texts[i], 2))
    if stars_un_one_hot[i] == 3:
        three_stars.append((train_texts[i], 3))
    if stars_un_one_hot[i] == 4:
        four_stars.append((train_texts[i], 4))
    if stars_un_one_hot[i] == 5:
        five_stars.append((train_texts[i], 5))

In [91]:
ones = models.tokenizer_sorted_out([i[0] for i in one_stars])
twos = models.tokenizer_sorted_out([i[0] for i in two_stars])
threes = models.tokenizer_sorted_out([i[0] for i in three_stars])
fours = models.tokenizer_sorted_out([i[0] for i in four_stars])
fives = models.tokenizer_sorted_out([i[0] for i in five_stars])

In [92]:
ones_sum = sum([i[1] for i in ones])
twos_sum = sum([i[1] for i in twos])
threes_sum = sum([i[1] for i in threes])
fours_sum = sum([i[1] for i in fours])
fives_sum = sum([i[1] for i in fives])

In [93]:
ones = [(i[0], i[1] / ones_sum) for i in ones]
twos = [(i[0], i[1] / twos_sum) for i in twos]
threes = [(i[0], i[1] / threes_sum) for i in threes]
fours = [(i[0], i[1] / fours_sum) for i in fours]
fives = [(i[0], i[1] / fives_sum) for i in fives]

In [94]:
ones = dict(ones)
twos = dict(twos)
threes = dict(threes)
fours = dict(fours)
fives = dict(fives)

In [95]:
def review_to_dict(entry):
    entry = entry.split()
    clean_punc = []
    for i in entry: 
        temp = i.translate(str.maketrans('', '', string.punctuation))
        if temp != "":
            clean_punc.append(temp.lower())
    entry = clean_punc
    entry_dict = dict()
    for i in entry:
        if i in entry_dict:
            entry_dict[i] = entry_dict[i] + 1
        else:
            entry_dict[i] = 1
    for i in entry_dict.keys():
        entry_dict[i] = entry_dict[i] / len(entry)
    return entry_dict

In [96]:
def kl_divergence(p, q):
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))
def comp_dicts(sdict, rdict):
    rdist = []
    sdist = []
    for i in rdict.keys():
        if i in sdict:
            rdist.append(rdict[i])
            sdist.append(sdict[i])
    return kl_divergence(np.array(rdist), np.array(sdist))

In [105]:
corr = 0
dist = 0
for i in range(2500):
    rdict = review_to_dict(test_texts[i])
    ind = np.argmin(np.array([np.abs(comp_dicts(ones, rdict)), np.abs(comp_dicts(twos, rdict)), 
                        np.abs(comp_dicts(threes, rdict)), np.abs(comp_dicts(fours, rdict)), 
                        np.abs(comp_dicts(fives, rdict))]))
    if ind == np.argmax(test_stars[i]):
        corr += 1
    else:
        dist += np.abs(ind - np.argmax(test_stars[i]))
print(corr / 2500)
print(dist / (2500 - corr))
print((1 - (corr / 2500)) + (dist / (2500 - corr)))

0.5084
1.614320585842148
2.1059205858421484
